In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb

In [6]:

df_merged = 'merged_shuffled'
# df_2 = pd.read_csv('../dataset/informative_cluster_2.csv')
selected_file = df_merged
df= pd.read_csv("../dataset/" + selected_file + ".csv")
df_fake = pd.read_csv("../dataset/fake_" + selected_file + ".csv")
files = [df_merged]

In [7]:
if 'Unnamed: 0' in df.columns:
    df= df.drop(columns=["Unnamed: 0"])
df.head()

,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest
0,1,205480.23,5949.01,0.0,1235745.12,1441225.36
1,1,167806.93,167806.93,0.0,0.00,167806.93
2,1,181330.91,37778.00,0.0,36793.35,218124.26
3,4,167907.12,167907.12,0.0,0.00,0.00
4,4,1434440.31,1434440.31,0.0,0.00,0.00


In [8]:
df_fake.head()

,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest
0,1,249120.14,2377531.77,814.36,27632.75,4406226.70
1,1,14785.81,42063.94,0.00,176827.96,1122000.98
2,3,2086.72,59208.48,3865.90,2404.70,30228.16
3,1,735091.18,71176.03,15203.80,682269.17,1134285.45
4,3,23479.01,123190.82,5494.57,12689.07,0.00


We can't calculate CorrelationSimilarity for minority clusters because there is totally constant values, type and origBalanceDist.
This metric measures the correlation between a pair of numerical columns and computes the similarity between the real and synthetic data

In [9]:
from sdmetrics.column_pairs import CorrelationSimilarity
correlation_similarities = []
temp_files = [df_merged]
for file in temp_files:
    df_org = pd.read_csv("../dataset/" + file + ".csv")
    if 'Unnamed: 0' in df_org.columns:
        df_org = df_org.drop(columns=["Unnamed: 0"])
    df_fake = pd.read_csv("../dataset/fake_" + file + ".csv")
    print(file)
    correlation_similarity = CorrelationSimilarity.compute(
    real_data=df_org[["amount","newbalanceDest"]],
    synthetic_data=df_fake[["amount","newbalanceDest"]],
    coefficient='Spearman'
    )
    print(correlation_similarity)
    correlation_similarities.append(correlation_similarity)


merged_shuffled
0.9978094711364953


KSComplement, This metric computes the similarity of a real column vs. a synthetic column in terms of the column shapes -- aka the marginal distribution or 1D histogram of the column.

In [12]:
from sdmetrics.single_column import KSComplement
correlation_similarities = []
column_scores = []
for file in files:
    df_org = pd.read_csv("../dataset/" + file + ".csv")
    if 'Unnamed: 0' in df_org.columns:
        df_org = df_org.drop(columns=["Unnamed: 0"])
    df_fake = pd.read_csv("../dataset/fake_" + file + ".csv")
    print(file)
    for column in df_org.columns:
        if column in ["type"]:
            continue
        ks_comp = KSComplement.compute(
        real_data=df_org[column],
        synthetic_data=df_fake[column],
        )
        column_scores.append(ks_comp)

    avg = sum(column_scores)/len(column_scores)
    print(avg)
    correlation_similarities.append(avg)
    column_scores = []


merged_shuffled
0.7291780905379882


StatisticSimilarity: This metric measures the similarity between a real column and a synthetic column by comparing a summary statistic. Supported summary statistics are: mean, median and standard deviation.


In [13]:
from sdmetrics.single_column import StatisticSimilarity
correlation_similarities = []
column_scores = []
for file in files:
    df_org = pd.read_csv("../dataset/" + file + ".csv")
    if 'Unnamed: 0' in df_org.columns:
        df_org = df_org.drop(columns=["Unnamed: 0"])
    df_fake = pd.read_csv("../dataset/fake_" + file + ".csv")
    print(file)
    for column in df_org.columns:
        if column in ["type"]:
            continue
        ks_comp = StatisticSimilarity.compute(
        real_data=df_org[column],
        synthetic_data=df_fake[column],
        )
        column_scores.append(ks_comp)

    avg = sum(column_scores)/len(column_scores)
    print(avg)
    correlation_similarities.append(avg)
    column_scores = []


merged_shuffled
0.9954317250008998


TVComplement:
This metric computes the similarity of a real column vs. a synthetic column in terms of the column shapes -- aka the marginal distribution or 1D histogram of the column.
Only for categorical data

In [8]:
from sdmetrics.single_column import TVComplement
correlation_similarities = []
for file in files:
    df_org = pd.read_csv("../dataset/" + file + ".csv")
    if 'Unnamed: 0' in df_org.columns:
        df_org = df_org.drop(columns=["Unnamed: 0"])
    df_fake = pd.read_csv("../dataset/fake_" + file + ".csv")
    print(file)
    ks_comp = TVComplement.compute(
    real_data=df_org["type"],
    synthetic_data=df_fake["type"],
    )

    print(ks_comp)


merged_shuffled
0.907085846585024


Privacy metrics

The DCRBaselineProtection metric measures the distance between your synthetic data and real data to measure how private it is. For a fair measurement, it compares the distance against randomly generated data, which would provide the best possible privacy protection.



In [14]:
from sdmetrics.single_table import DCRBaselineProtection
from sdv.metadata import Metadata

correlation_similarities = []
for file in files:
    df_org = pd.read_csv("../dataset/" + file + ".csv")
    if 'Unnamed: 0' in df_org.columns:
        df_org = df_org.drop(columns=["Unnamed: 0"])
    df_fake = pd.read_csv("../dataset/fake_" + file + ".csv")
    print(file)
    ks_comp = DCRBaselineProtection.compute_breakdown(
    real_data=df_org,
    synthetic_data=df_fake,
    metadata= Metadata.detect_from_dataframe(df_org).to_dict().get("tables").get("table")
    )

    print(ks_comp['score'])


merged_shuffled
0.001826399455942299


The DisclosureProtection metric measures the risk associated with disclosing (aka broadly sharing) the synthetic data. It's a useful measurement if you want to know whether synthetic data is leaking patterns that pertain to sensitive information.

In [15]:
from sdmetrics.single_table import DisclosureProtection
from sdv.metadata import Metadata

correlation_similarities = []
for file in files:
    df_org = pd.read_csv("../dataset/" + file + ".csv")
    if 'Unnamed: 0' in df_org.columns:
        df_org = df_org.drop(columns=["Unnamed: 0"])
    df_fake = pd.read_csv("../dataset/fake_" + file + ".csv")
    print(file)
    ks_comp = DisclosureProtection.compute(
    real_data=df_org,
    synthetic_data=df_fake,
    known_column_names=['newbalanceOrig','newbalanceDest','oldbalanceDest'],
    sensitive_column_names=['amount'],
    )

    print(ks_comp)


merged_shuffled


I:\ETVOS\2nd semester\DS Lab 1\DSLAB-Project\venv1\Lib\site-packages\sdmetrics\single_table\privacy\disclosure_protection.py:214: UserWarning: Data exceeds 10000 rows, perfomance may be slow. Consider using the `DisclosureProtectionEstimate` for faster computation.
  warnings.warn(


1


In [18]:
from sdmetrics.single_table import DisclosureProtection
from sdmetrics.single_table.data_augmentation import BinaryClassifierRecallEfficacy
from sdv.metadata import SingleTableMetadata
import pandas as pd
from sklearn.model_selection import train_test_split



df_merged = 'merged_shuffled_labeled'
# df_2 = pd.read_csv('../dataset/informative_cluster_2.csv')
selected_file = df_merged
df= pd.read_csv("../dataset/" + selected_file + ".csv")
df_fake = pd.read_csv("../dataset/fake_" + selected_file + ".csv")

efficacy_scores = []


# Step 1: Train-test split from real data for validation
train_set, real_holdout_set = train_test_split(df, test_size=0.2, stratify=df["isFraud"], random_state=42)

# Step 2: Build metadata (you can also load it from a file if needed)
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(train_set)

# Step 3: Compute Efficacy score
score = BinaryClassifierRecallEfficacy.compute_breakdown(
    real_training_data=train_set,
    synthetic_data=df_fake,
    real_validation_data=real_holdout_set,
    metadata=metadata.to_dict(),
    prediction_column_name="isFraud",
    minority_class_label=1,
    classifier="XGBoost",
    fixed_precision_value=0.9
)

print(score["score"])
efficacy_scores.append(score)


0.49939135727328055


In [21]:
from sdmetrics.single_table import DisclosureProtection
from sdmetrics.single_table.data_augmentation import BinaryClassifierPrecisionEfficacy
from sdv.metadata import SingleTableMetadata
import pandas as pd
from sklearn.model_selection import train_test_split



df_merged = 'merged_shuffled_labeled'
# df_2 = pd.read_csv('../dataset/informative_cluster_2.csv')
selected_file = df_merged
df= pd.read_csv("../dataset/" + selected_file + ".csv")
df_fake = pd.read_csv("../dataset/fake_" + selected_file + ".csv")

efficacy_scores = []


# Step 1: Train-test split from real data for validation
train_set, real_holdout_set = train_test_split(df, test_size=0.2, stratify=df["isFraud"], random_state=42)

# Step 2: Build metadata (you can also load it from a file if needed)
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(train_set)

# Step 3: Compute Efficacy score
score = BinaryClassifierPrecisionEfficacy.compute_breakdown(
    real_training_data=train_set,
    synthetic_data=df_fake,
    real_validation_data=real_holdout_set,
    metadata=metadata.to_dict(),
    prediction_column_name="isFraud",
    minority_class_label=1,
    classifier="XGBoost",
    fixed_recall_value=0.9
)

print(score["score"])
efficacy_scores.append(score)


0.4981640146878825
